In [2]:
import numpy as np
import pandas as pd

# Step 1: Set a random seed for reproducibility
np.random.seed(42)

# Step 2: Define parameters
n_samples = 500      # number of data points
n_features = 7       # number of features

# Step 3: Create a covariance matrix to make features correlated
base_corr = 0.9
cov = np.full((n_features, n_features), base_corr)
np.fill_diagonal(cov, 1.0)  # diagonal = 1 (self-correlation)

# Step 4: Generate multivariate normal data (highly correlated)
mean = np.zeros(n_features)
X = np.random.multivariate_normal(mean, cov, size=n_samples)

# Step 5: Define true coefficients and bias
true_weights = np.array([2.5, -1.8, 1.2, 0.8, 0.5, 1.5, -0.7])
bias = 3.0

# Step 6: Generate target variable with some noise They define how the target variable (y) 
#is generated from your features (X) using a linear relationship + randomness
noise = np.random.normal(0, 1.5, size=n_samples)
y = X.dot(true_weights) + bias + noise

# Step 7: Create DataFrame
columns = [f'Feature_{i+1}' for i in range(n_features)]
df = pd.DataFrame(X, columns=columns)
df['Target'] = y

# Step 8: Display correlation matrix
print("Feature Correlation Matrix:")
print(df.corr())

# Step 9: Save dataset
df.to_csv("highly_correlated_dataset.csv", index=False)
print("\nDataset saved as 'highly_correlated_dataset.csv'")


Feature Correlation Matrix:
           Feature_1  Feature_2  Feature_3  Feature_4  Feature_5  Feature_6  \
Feature_1   1.000000   0.897950   0.889118   0.903796   0.896104   0.902124   
Feature_2   0.897950   1.000000   0.891180   0.906176   0.906239   0.898228   
Feature_3   0.889118   0.891180   1.000000   0.904157   0.905161   0.898464   
Feature_4   0.903796   0.906176   0.904157   1.000000   0.892401   0.911539   
Feature_5   0.896104   0.906239   0.905161   0.892401   1.000000   0.901148   
Feature_6   0.902124   0.898228   0.898464   0.911539   0.901148   1.000000   
Feature_7   0.895656   0.906879   0.899301   0.911495   0.894585   0.904232   
Target      0.897466   0.788642   0.864259   0.863332   0.846861   0.877392   

           Feature_7    Target  
Feature_1   0.895656  0.897466  
Feature_2   0.906879  0.788642  
Feature_3   0.899301  0.864259  
Feature_4   0.911495  0.863332  
Feature_5   0.894585  0.846861  
Feature_6   0.904232  0.877392  
Feature_7   1.000000  0.81478

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score

# Load dataset
url =r"C:\Users\96597\Downloads\Hitters (1).csv"
df = pd.read_csv(url)

# Preprocessing
df = df.dropna()
for col in df.select_dtypes('object'):
    df[col] = LabelEncoder().fit_transform(df[col])

X = df.drop('Salary', axis=1)
y = df['Salary']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Models
models = {
    "Linear": LinearRegression(),
    "Ridge": Ridge(alpha=0.5748),
    "Lasso": Lasso(alpha=0.5748)
}

for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    print(f"{name} R2 Score: {r2_score(y_test, preds):.4f}")


Linear R2 Score: 0.2907
Ridge R2 Score: 0.2998
Lasso R2 Score: 0.2991


C:\Users\96597\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 12475.051482647657, tolerance: 4367.0488194499285
  model = cd_fast.enet_coordinate_descent(


In [8]:
from sklearn.datasets import load_boston
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Load Boston dataset
X, y = load_boston(return_X_y=True)
X = StandardScaler().fit_transform(X)

# RidgeCV and LassoCV
ridge_cv = RidgeCV(alphas=[0.1,1,10], cv=5)
ridge_cv.fit(X, y)

lasso_cv = LassoCV(alphas=[0.1,1,10], cv=5)
lasso_cv.fit(X, y)

print(f"Best Ridge alpha: {ridge_cv.alpha_}")
print(f"Best Lasso alpha: {lasso_cv.alpha_}")


Best Ridge alpha: 10.0
Best Lasso alpha: 0.1


In [9]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

iris = load_iris()
X, y = iris.data, iris.target

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step-by-step Logistic Regression (One-vs-Rest)
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def logistic_regression(X, y, lr=0.1, epochs=1000):
    m, n = X.shape
    theta = np.zeros((n,1))
    y = y.reshape(-1,1)
    for _ in range(epochs):
        preds = sigmoid(X @ theta)
        grad = (1/m) * X.T @ (preds - y)
        theta -= lr * grad
    return theta

# Train OVR
thetas = []
for k in np.unique(y_train):
    y_k = (y_train == k).astype(int)
    thetas.append(logistic_regression(X_train, y_k))

# Predict
preds = []
for x in X_test:
    scores = [sigmoid(x @ theta)[0] for theta in thetas]
    preds.append(np.argmax(scores))

print(f"Accuracy: {accuracy_score(y_test, preds)*100:.2f}%")


Accuracy: 90.00%
